In [1]:
pip install numpy pandas scikit-learn tensorflow matplotlib seaborn jupyterlab torch torch-geometric rdkit

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# === Core Libraries ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import glob

# === Progress Bar ===
from tqdm.keras import TqdmCallback

# === Scikit-learn ===
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# === TensorFlow / Keras ===
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, Conv1D, GlobalMaxPooling1D,
    Dense, Concatenate, BatchNormalization, Dropout
)
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import Callback, ModelCheckpoint


c:\Users\NongNam\Documents\AI_Builder\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv('Kd.csv')

In [6]:
df.isna().sum()

SMILES             0
target_sequence    0
Kd                 0
dtype: int64

In [5]:
len(df)

55011

In [7]:
# 3. Encode SMILES
smiles_chars = sorted(set(''.join(df['SMILES'])))
char_to_int_smiles = {c: i + 1 for i, c in enumerate(smiles_chars)}

def encode_smiles(smiles):
    return [char_to_int_smiles[c] for c in smiles]

df['smiles_encoded'] = df['SMILES'].apply(encode_smiles)

# 4. Encode protein sequences
aa_list = 'ACDEFGHIKLMNPQRSTVWY'
aa_to_int = {aa: i + 1 for i, aa in enumerate(aa_list)}

def encode_protein(seq):
    return [aa_to_int.get(aa, 0) for aa in seq]

df['protein_encoded'] = df['target_sequence'].apply(encode_protein)

In [9]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import numpy as np

In [13]:
# --- Molecular descriptors from SMILES ---
def compute_molecular_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [None] * 5
    return [
        Descriptors.MolWt(mol),
        Descriptors.MolLogP(mol),
        Descriptors.NumRotatableBonds(mol),
        Descriptors.NumHDonors(mol),
        Descriptors.NumHAcceptors(mol)
    ]

df[['MolWt', 'LogP', 'RotatableBonds', 'HDonors', 'HAcceptors']] = df['SMILES'].apply(
    compute_molecular_descriptors
).apply(pd.Series)

# --- Protein descriptors from sequence ---
def compute_protein_descriptors(seq):
    try:
        analyzed_seq = ProteinAnalysis(seq)
        return [
            analyzed_seq.molecular_weight(),
            analyzed_seq.aromaticity(),
            analyzed_seq.instability_index(),
            analyzed_seq.gravy()
        ]
    except:
        return [None] * 4
    
df[['Prot_MW', 'Aromaticity', 'Instability', 'Hydropathy']] = df['target_sequence'].apply(
    compute_protein_descriptors
).apply(pd.Series)

[19:51:55] Explicit valence for atom # 22 N, 4, is greater than permitted
[19:51:58] Explicit valence for atom # 22 N, 4, is greater than permitted
[19:52:02] Explicit valence for atom # 22 N, 4, is greater than permitted
[19:52:11] Explicit valence for atom # 22 N, 4, is greater than permitted
[19:52:27] Explicit valence for atom # 22 N, 4, is greater than permitted
[19:52:39] Can't kekulize mol.  Unkekulized atoms: 16 17 18 19 20 21 22 23 25
[19:52:48] Explicit valence for atom # 22 N, 4, is greater than permitted


In [14]:
df = df.dropna(subset=[
    'MolWt', 'LogP', 'RotatableBonds', 'HDonors', 'HAcceptors',
    'Prot_MW', 'Aromaticity', 'Instability', 'Hydropathy'
])


In [33]:
x_descriptor = df[['MolWt', 'LogP', 'RotatableBonds', 'HDonors', 'HAcceptors']].values
X_protein_descriptor = df[['Prot_MW', 'Aromaticity', 'Instability', 'Hydropathy']].values


In [21]:
# 5. Pad sequences
max_len_smiles = 100
max_len_protein = 1000

X_smiles = pad_sequences(df['smiles_encoded'], maxlen=max_len_smiles, padding='post')
X_protein = pad_sequences(df['protein_encoded'], maxlen=max_len_protein, padding='post')
y = df['Kd'].values

In [35]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, Conv1D, GlobalMaxPooling1D,
    Dense, Dropout, BatchNormalization, Concatenate
)
from tensorflow.keras.losses import MeanSquaredError

# --- Inputs ---
# 1. SMILES token branch
smiles_input = Input(shape=(max_len_smiles,), name='smiles_input')
x1 = Embedding(input_dim=len(char_to_int_smiles) + 1, output_dim=64)(smiles_input)
x1 = Conv1D(128, kernel_size=5, activation='relu', padding='same')(x1)
x1 = BatchNormalization()(x1)
x1 = Conv1D(128, kernel_size=5, activation='relu', padding='same')(x1)
x1 = GlobalMaxPooling1D()(x1)
x1 = Dropout(0.3)(x1)

# 2. Protein token branch
protein_input = Input(shape=(max_len_protein,), name='protein_input')
x2 = Embedding(input_dim=len(aa_to_int) + 1, output_dim=64)(protein_input)
x2 = Conv1D(128, kernel_size=9, activation='relu', padding='same')(x2)
x2 = BatchNormalization()(x2)
x2 = Conv1D(128, kernel_size=9, activation='relu', padding='same')(x2)
x2 = GlobalMaxPooling1D()(x2)
x2 = Dropout(0.3)(x2)

# 3. Molecular descriptors branch (5 features)
mol_desc_input = Input(shape=(5,), name='mol_desc_input')
x3 = Dense(32, activation='relu')(mol_desc_input)
x3 = BatchNormalization()(x3)

# 4. Protein descriptors branch (4 features)
prot_desc_input = Input(shape=(4,), name='prot_desc_input')
x4 = Dense(32, activation='relu')(prot_desc_input)
x4 = BatchNormalization()(x4)

# --- Merge all branches ---
merged = Concatenate()([x1, x2, x3, x4])

# --- Fully connected layers ---
x = Dense(256, activation='relu')(merged)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)

# --- Output ---
output = Dense(1, activation='linear', name='kd_output')(x)

# --- Model ---
model = Model(inputs=[smiles_input, protein_input, mol_desc_input, prot_desc_input], outputs=output)
model.compile(optimizer='adam', loss=MeanSquaredError(), metrics=['mae'])

# --- Summary ---
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ smiles_input        │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ protein_input       │ (None, 1000)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 100, 64)   │      2,688 │ smiles_input[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, 1000, 64)  │      1,344 │ protein_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 100, 128)  │     41,088 │ embedding_4[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_10 (Conv1D)  │ (None, 1000, 128) │     73,856 │ embedding_5[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 100, 128)  │        512 │ conv1d_8[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1000, 128) │        512 │ conv1d_10[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_9 (Conv1D)   │ (None, 100, 128)  │     82,048 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_11 (Conv1D)  │ (None, 1000, 128) │    147,584 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mol_desc_input      │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prot_desc_input     │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_9[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_11[0][0]   │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 32)        │        192 │ mol_desc_input[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 32)        │        160 │ prot_desc_input[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 128)       │          0 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 128)       │          0 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32)        │        128 │ dense_7[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 466,465 (1.78 MB)

 Trainable params: 465,313 (1.78 MB)

 Non-trainable params: 1,152 (4.50 KB)

In [23]:
len(df)

54980

In [24]:
class EpochHistorySaver(Callback):
    def __init__(self, save_dir='SE+CN_MODEL'):
        super().__init__()
        self.save_dir = save_dir
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        for key, value in logs.items():
            self.history.setdefault(key, []).append(value)

        # Save history after each epoch
        filename = f'{self.save_dir}/history_epoch_{epoch+1:02d}.pkl'
        with open(filename, 'wb') as f:
            pickle.dump(self.history, f)


In [ ]:
checkpoint_cb = ModelCheckpoint(
    filepath='SE+CN_MODEL/model_epoch_{epoch:02d}.h5',
    save_freq='epoch',
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)

history_cb = EpochHistorySaver(save_dir='SE+CN_MODEL')


In [ ]:
# from tensorflow.keras.models import load_model
# model = load_model('CNN_MODEL/model_epoch_59.h5', compile=False)

In [13]:
# model.compile(optimizer='adam', loss=MeanSquaredError(), metrics=['mae'])

In [36]:
# 8. Train with progress bar
history = model.fit(
    [X_smiles, X_protein, x_descriptor, X_protein_descriptor],
    y,
    validation_split=0.22,
    epochs=100,
    batch_size=64,
    callbacks=[checkpoint_cb, history_cb]
)

Epoch 1/100
670/671 ━━━━━━━━━━━━━━━━━━━━ 0s 592ms/step - loss: 7.7446 - mae: 2.0826
Epoch 1: saving model to SE+CN_MODELl/model_epoch_01.h5


671/671 ━━━━━━━━━━━━━━━━━━━━ 422s 621ms/step - loss: 7.7342 - mae: 2.0810 - val_loss: 7.6846 - val_mae: 2.3893
Epoch 2/100
670/671 ━━━━━━━━━━━━━━━━━━━━ 0s 561ms/step - loss: 2.4542 - mae: 1.2020
Epoch 2: saving model to SE+CN_MODELl/model_epoch_02.h5


671/671 ━━━━━━━━━━━━━━━━━━━━ 394s 587ms/step - loss: 2.4537 - mae: 1.2018 - val_loss: 3.2784 - val_mae: 1.4422
Epoch 3/100
670/671 ━━━━━━━━━━━━━━━━━━━━ 0s 512ms/step - loss: 1.9882 - mae: 1.0759
Epoch 3: saving model to SE+CN_MODELl/model_epoch_03.h5


671/671 ━━━━━━━━━━━━━━━━━━━━ 361s 538ms/step - loss: 1.9882 - mae: 1.0759 - val_loss: 2.2338 - val_mae: 1.1618
Epoch 4/100
670/671 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step - loss: 1.8227 - mae: 1.0188
Epoch 4: saving model to SE+CN_MODELl/model_epoch_04.h5


671/671 ━━━━━━━━━━━━━━━━━━━━ 343s 511ms/step - loss: 1.8225 - mae: 1.0188 - val_loss: 1.7219 - val_mae: 0.9448
Epoch 5/100
670/671 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step - loss: 1.6557 - mae: 0.9729
Epoch 5: saving model to SE+CN_MODELl/model_epoch_05.h5


...

671/671 ━━━━━━━━━━━━━━━━━━━━ 372s 555ms/step - loss: 0.4700 - mae: 0.4735 - val_loss: 0.6940 - val_mae: 0.5331
Epoch 97/100
670/671 ━━━━━━━━━━━━━━━━━━━━ 0s 531ms/step - loss: 0.4660 - mae: 0.4765
Epoch 97: saving model to SE+CN_MODELl/model_epoch_97.h5


671/671 ━━━━━━━━━━━━━━━━━━━━ 373s 556ms/step - loss: 0.4660 - mae: 0.4765 - val_loss: 0.6941 - val_mae: 0.5329
Epoch 98/100
670/671 ━━━━━━━━━━━━━━━━━━━━ 0s 531ms/step - loss: 0.4630 - mae: 0.4699
Epoch 98: saving model to SE+CN_MODELl/model_epoch_98.h5


671/671 ━━━━━━━━━━━━━━━━━━━━ 373s 556ms/step - loss: 0.4630 - mae: 0.4699 - val_loss: 0.7351 - val_mae: 0.5402
Epoch 99/100
670/671 ━━━━━━━━━━━━━━━━━━━━ 0s 532ms/step - loss: 0.4647 - mae: 0.4695
Epoch 99: saving model to SE+CN_MODELl/model_epoch_99.h5


671/671 ━━━━━━━━━━━━━━━━━━━━ 374s 558ms/step - loss: 0.4647 - mae: 0.4695 - val_loss: 0.6983 - val_mae: 0.5330
Epoch 100/100
670/671 ━━━━━━━━━━━━━━━━━━━━ 0s 532ms/step - loss: 0.4734 - mae: 0.4787
Epoch 100: saving model to SE+CN_MODELl/model_epoch_100.h5


671/671 ━━━━━━━━━━━━━━━━━━━━ 374s 558ms/step - loss: 0.4734 - mae: 0.4787 - val_loss: 0.6926 - val_mae: 0.5359


In [ ]:
# Load all history files
history_files = sorted(glob.glob('SE+CN_MODEL/history_epoch_*.pkl'))

full_history = {}
for file in history_files:
    with open(file, 'rb') as f:
        epoch_history = pickle.load(f)
    for key, values in epoch_history.items():
        full_history.setdefault(key, []).extend(values[len(full_history.get(key, [])):])

# Plot
import matplotlib.pyplot as plt

plt.plot(full_history['loss'])
plt.plot(full_history['val_loss'])
plt.title('Model accuracy over all epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])
plt.show()